# Only Local Vars Decorator

The purpose of this notebook is to show how to create a decorator to ensure a function only uses local variables. This ensures that the function does not accidentally use global or nonlocal variables from the global scope, which can avoid many common bugs.

Important notes:

* Global variables are variables defined in the global scope that can be read from anywhere in the file after the variable is created

* Nonlocal variables only exist in the context of nested functions, in which we have an outer function and an inner function defined within the outer function. From within the inner function, the variables defined in the outer function can be read, and these are called nonlocal variables.

## Create `only_local_vars` Decorator

In [91]:
import inspect
import types

def is_variable(x):
    is_module = isinstance(x, types.ModuleType)
    is_function = hasattr(x, '__call__')
    return not is_module and not is_function

def only_local_vars(f):
    def check_for_global_and_nonlocal_vars(*args, **kwargs):
        # Check for nonlocal and global variables
        closure_vars = inspect.getclosurevars(f)
        nonlocal_vars = {name: val for name, val in closure_vars.nonlocals.items() if is_variable(val)}
        global_vars = {name: val for name, val in closure_vars.globals.items() if is_variable(val)}
        
        # Assertions
        if len(nonlocal_vars) > 0:
            raise AssertionError(f"The function '{f.__name__}' should not be using the following nonlocal vars: {nonlocal_vars}")
        if len(global_vars) > 0:
            raise AssertionError(f"The function '{f.__name__}' should not be using the following global vars: {global_vars}")
        
        # Run function
        return f(*args, **kwargs)
    return check_for_global_and_nonlocal_vars

## Test `only_local_vars`

In [92]:
# BEST
@only_local_vars
def test_only_local_vars_GOOD(x, repeat):
    return np.array([x]*repeat)

@only_local_vars
def test_only_local_vars_typo_GOOD(x_typo, repeat_typo):
    return np.array([x]*repeat)

@only_local_vars
def test_only_local_vars_nested_GOOD(x, repeat=10):
    @only_local_vars
    def helper(x, repeat):
        return np.array([x]*repeat)
    return helper(x, repeat)

@only_local_vars
def test_only_local_vars_nested_typo_GOOD(x, repeat=10):
    @only_local_vars
    def helper(x_typo, repeat_typo):
        return np.array([x]*repeat)
    return helper(x, repeat)

##  Create Alternative that Fail

In [93]:
# BAD
def test_baseline_GOOD(x, repeat):
    return np.array([x]*repeat)

def test_baseline_typo_BAD(x_typo, repeat_typo):
    return np.array([x]*repeat)

def test_baseline_nested_GOOD(x, repeat=10):
    def helper(x, repeat):
        return np.array([x]*repeat)
    return helper(x, repeat)

def test_baseline_nested_typo_BAD(x, repeat=10):
    def helper(x_typo, repeat_typo):
        return np.array([x]*repeat)
    return helper(x, repeat)

In [94]:
# Create global variables with the same variable name
x = 1
repeat = 2

## Show working `only_local_vars`

In [95]:
# GOOD
print("GOOD: CORRECT OUTPUT")
print(f"test_only_local_vars_GOOD(x, repeat) = {test_only_local_vars_GOOD(x, repeat)}")
print(f"test_only_local_vars_GOOD(5, 10) = {test_only_local_vars_GOOD(5, 10)}")

GOOD: CORRECT OUTPUT
test_only_local_vars_GOOD(x, repeat) = [1 1]
test_only_local_vars_GOOD(5, 10) = [5 5 5 5 5 5 5 5 5 5]


In [96]:
# GOOD
print("GOOD: ERROR FROM TYPO")
print(f"test_only_local_vars_typo_GOOD(x, repeat) = {test_only_local_vars_typo_GOOD(x, repeat)}")
print(f"test_only_local_vars_typo_GOOD(5, 10) = {test_only_local_vars_typo_GOOD(5, 10)}")

GOOD: ERROR FROM TYPO


AssertionError: The function 'test_only_local_vars_typo_GOOD' should not be using the following global vars from : {'x': 1, 'repeat': 2}

In [97]:
# GOOD
print("GOOD: NESTED FUNCTION WORKS")
print(f"test_only_local_vars_nested_GOOD(x, repeat) = {test_only_local_vars_nested_GOOD(x, repeat)}")
print(f"test_only_local_vars_nested_GOOD(5, 10) = {test_only_local_vars_nested_GOOD(5, 10)}")

GOOD: NESTED FUNCTION WORKS
test_only_local_vars_nested_GOOD(x, repeat) = [1 1]
test_only_local_vars_nested_GOOD(5, 10) = [5 5 5 5 5 5 5 5 5 5]


In [98]:
# GOOD
print("GOOD: ERROR FROM NONLOCAL VARIABLE TYPO")
print(f"test_only_local_vars_nested_typo_GOOD(x, repeat) = {test_only_local_vars_nested_typo_GOOD(x, repeat)}")
print(f"test_only_local_vars_nested_typo_GOOD(5, 10) = {test_only_local_vars_nested_typo_GOOD(5, 10)}")

GOOD: ERROR FROM NONLOCAL VARIABLE TYPO


AssertionError: The function 'helper' should not be using the following nonlocal vars from : {'repeat': 2, 'x': 1}

## Show not working `baseline`

In [99]:
# GOOD
print("GOOD: CORRECT OUTPUT")
print(f"test_baseline_GOOD(x, repeat) = {test_baseline_GOOD(x, repeat)}")
print(f"test_baseline_GOOD(5, 10) = {test_baseline_GOOD(5, 10)}")

GOOD: CORRECT OUTPUT
test_baseline_GOOD(x, repeat) = [1 1]
test_baseline_GOOD(5, 10) = [5 5 5 5 5 5 5 5 5 5]


In [100]:
# BAD
print("BAD: SUBTLE BUG FROM TYPO")
print(f"test_baseline_typo_BAD(x, repeat) = {test_baseline_typo_BAD(x, repeat)}")
print(f"test_baseline_typo_BAD(5, 10) = {test_baseline_typo_BAD(5, 10)}")

BAD: SUBTLE BUG FROM TYPO
test_baseline_typo_BAD(x, repeat) = [1 1]
test_baseline_typo_BAD(5, 10) = [1 1]


In [101]:
# GOOD
print("GOOD: CORRECT OUTPUT")
print(f"test_baseline_nested_GOOD(x, repeat) = {test_baseline_nested_GOOD(x, repeat)}")
print(f"test_baseline_nested_GOOD(5, 10) = {test_baseline_nested_GOOD(5, 10)}")

GOOD: CORRECT OUTPUT
test_baseline_nested_GOOD(x, repeat) = [1 1]
test_baseline_nested_GOOD(5, 10) = [5 5 5 5 5 5 5 5 5 5]


In [102]:
# BAD
print("BAD: SUBTLE BUG FROM TYPO")
print(f"test_baseline_nested_typo_BAD(x, repeat) = {test_baseline_nested_typo_BAD(x, repeat)}")
print(f"test_baseline_nested_typo_BAD(5, 10) = {test_baseline_nested_typo_BAD(5, 10)}")

BAD: SUBTLE BUG FROM TYPO
test_baseline_nested_typo_BAD(x, repeat) = [1 1]
test_baseline_nested_typo_BAD(5, 10) = [5 5 5 5 5 5 5 5 5 5]
